In [10]:
import sys,os
sys.path.append("..")
import django
django.setup()
from crawlers.models import *
import pandas as pd
import datetime
from crawlers.finlab.import_tools import *
from components.backend_api import DataFilter
import numpy as np
from talib import abstract

In [ ]:
test=DataFilter(StockDivideRatioTW,['stock_id','date','divide_ratio'])
test.get_pivot()

In [90]:
from components.backend_api import DataFilter
import numpy as np
from crawlers.models import *


class GetAdj(DataFilter):

    @staticmethod
    def adj_holiday(item, df):
        all_index = (df.index | item.index).sort_values()
        all_index = all_index[all_index >= item.index[0]]
        df = df.reindex(all_index)
        group = all_index.isin(item.index).cumsum()
        df = df.groupby(group).mean()
        df.index = item.index
        return df

    def get(self):
        columns = ['stock_id', 'date', self.fields]
        query_set = {'model': self.model, 'fields': columns, 'offset': self.offset, 'limit': self.limit,
                     'start_date': self.start_date, 'end_date': self.end_date}
        price = DataFilter(**query_set)
        item = price.get_pivot()
        divide_ratio = DataFilter(StockDivideRatioTW, ['stock_id', 'date', 'divide_ratio'])
        ratio = self.adj_holiday(item, divide_ratio.get_pivot())
        divide_ratio = (ratio.reindex_like(item).fillna(1).cumprod())
        divide_ratio[np.isinf(divide_ratio)] = 1
        result = (item * divide_ratio)
        return result

In [91]:
df=GetAdj(model=StockPriceTW,fields='close_price',offset=1000).get()
df

stock_id,0050,0051,0052,0053,0054,0055,0056,0057,0058,0059,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2016-06-13,63.750000,24.180000,37.930000,27.330000,20.140000,12.820000,22.060000,39.190000,36.05,29.54,...,36.900000,13.400000,11.30,18.250000,165.500000,12.15,6.60,9.830000,21.700000,8.240000
2016-06-14,64.150000,24.390000,38.260000,27.440000,20.100000,12.790000,22.280000,39.070000,36.14,29.85,...,37.300000,13.400000,11.35,18.150000,164.000000,12.35,6.71,9.760000,21.600000,8.450000
2016-06-15,64.250000,24.430000,38.260000,27.550000,20.330000,12.710000,22.270000,39.270000,36.08,NaN,...,37.900000,13.400000,11.40,18.000000,165.500000,12.65,6.71,9.880000,21.650000,8.530000
2016-06-16,63.350000,24.190000,38.490000,27.180000,19.980000,12.560000,22.080000,38.500000,36.13,29.83,...,37.350000,13.500000,11.40,17.850000,165.000000,12.75,6.73,9.870000,21.600000,8.510000
2016-06-17,64.050000,24.230000,38.590000,27.320000,20.230000,12.650000,22.180000,39.220000,36.28,29.83,...,36.500000,13.450000,11.45,17.700000,164.500000,12.50,6.73,9.860000,21.700000,8.540000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-07,109.691237,43.978715,83.820565,50.992066,29.459155,20.898198,36.348420,65.574849,NaN,NaN,...,48.832268,21.006077,13.05,10.348031,90.876619,20.60,5.10,88.235385,29.593467,12.110462
2020-07-08,110.445727,44.159605,84.715844,51.565011,29.618825,20.955453,36.706594,65.626892,NaN,NaN,...,49.192654,20.937429,13.00,10.292989,90.758751,22.15,5.10,89.318028,29.593467,11.358183
2020-07-09,110.793953,44.555300,85.163484,51.759812,29.653040,20.932551,36.731295,65.991197,NaN,NaN,...,48.922365,21.006077,13.00,10.315006,88.872854,21.90,5.12,90.617200,29.646031,11.444510


In [96]:
a=['stock_id','date','2','3']
a=[i for i in a if i not in ['stock_id','date']]
a

['2', '3']

In [4]:
columns = ['stock_id', 'date', 'close_price']
price = DataFilter(StockPriceTW, columns, 1000)
df=price.get_pivot()
df

stock_id,0050,0051,0052,0053,0054,0055,0056,0057,0058,0059,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2016-06-13,63.75,24.18,37.93,27.33,20.14,12.82,22.06,39.19,36.05,29.54,...,36.90,13.40,11.30,18.25,165.5,12.15,6.60,9.83,21.70,8.24
2016-06-14,64.15,24.39,38.26,27.44,20.10,12.79,22.28,39.07,36.14,29.85,...,37.30,13.40,11.35,18.15,164.0,12.35,6.71,9.76,21.60,8.45
2016-06-15,64.25,24.43,38.26,27.55,20.33,12.71,22.27,39.27,36.08,NaN,...,37.90,13.40,11.40,18.00,165.5,12.65,6.71,9.88,21.65,8.53
2016-06-16,63.35,24.19,38.49,27.18,19.98,12.56,22.08,38.50,36.13,29.83,...,37.35,13.50,11.40,17.85,165.0,12.75,6.73,9.87,21.60,8.51
2016-06-17,64.05,24.23,38.59,27.32,20.23,12.65,22.18,39.22,36.28,29.83,...,36.50,13.45,11.45,17.70,164.5,12.50,6.73,9.86,21.70,8.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-07,94.50,38.90,74.90,44.50,25.83,18.25,29.43,63.00,NaN,NaN,...,54.20,15.30,13.05,9.40,77.1,20.60,5.10,81.50,28.15,9.82
2020-07-08,95.15,39.06,75.70,45.00,25.97,18.30,29.72,63.05,NaN,NaN,...,54.60,15.25,13.00,9.35,77.0,22.15,5.10,82.50,28.15,9.21
2020-07-09,95.45,39.41,76.10,45.17,26.00,18.28,29.74,63.40,NaN,NaN,...,54.30,15.30,13.00,9.37,75.4,21.90,5.12,83.70,28.20,9.28


In [7]:
divide_ratio = DataFilter(StockDivideRatioTW, ['stock_id', 'date', 'divide_ratio']).get_pivot()
divide_ratio

stock_id,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2004-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
